In [187]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import yfinance as yf
import datetime

import warnings
warnings.filterwarnings("ignore")

import RECH_functions as RECH

import pickle

In [188]:
act_func = RECH.relu

In [190]:
my_list = ["MMM",
"AXP",
"AMGN",
"AAPL",
"BA",
"CAT",
"CVX",
"CSCO",
"KO",
"DOW",
"GS",
"HD",
"HON",
"INTC",
"IBM",
"JNJ",
"JPM",
"MCD",
"MRK",
"MSFT",
"NKE",
"PG",
"CRM",
"TRV",
"UNH",
"VZ",
"V",
"WBA",
"WMT",
"DIS",
"^GSPC"]
my_list.remove("DOW") # dow joined in 2019
my_list.sort()

In [191]:
#my_list = ["IBM", "AAPL"]

In [192]:
#my_list = ["PG"]

In [193]:
#forecasts_symbol = pd.DataFrame(index=os_data.index) # empty dataframe to be filles with forecasts of different horizons
change_par = 0.2

vP0 = (0.1, 0.8, 0.1, 0.1 , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)
my_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-10, 10), (-10, 10), (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-100,100), (-100,100), (-100,100), (-100,100))

par_names = ["alpha", "beta", "gamma_0", "gamma_1", "v_11", "v_12", "v_21", "v_22", "v_31", "v_32", "v_41", "v_42", "w_1", "w_2", "w_3", "w_4", "b_c", "b_o", "b_i", "b_f"]

forecast_all = pd.DataFrame()

def con(t):
    return (-1)*(t[0] + t[1]) + 0.99
cons = {'type':'ineq', 'fun': con}

df_pars = pd.DataFrame(columns = par_names, index = my_list)
mc_M = 5000

act_func = RECH.relu
warning_list = []
for symbol in my_list:
    pd_this_share = 
    is_list = [(pd_this_share.index[x] < datetime.date(2017, 1, 1)) for x in range(len(pd_this_share)) ]
    is_data = pd_this_share[is_list]
    is_data.drop(index=is_data.index[0], axis=0, inplace=True) # dropping the first value with NA in returns
    is_returns = is_data['log_ret * 100']
    print("####################")
    print(is_returns.head(4))
    print("####################")
    # out of sample data is all the data from 01.01.2015
    os_list = [(pd_this_share.index[x] >= datetime.date(2017, 1, 1)) for x in range(len(pd_this_share)) ]
    os_data = pd_this_share[os_list]
    os_returns = os_data['log_ret * 100']
    vP0 = (0.1, 0.8, 0.1, 0.1 , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)
    my_bounds = ((0.0001,1), (0.0001,1), (0.0001, 10),  (0.0001, 3) , (-10, 10), (-10, 10), (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-10,10), (-10, 10) , (-10, 10), (-100,100), (-100,100), (-100,100), (-100,100))
    
    update_window = 20 # length of the updating window 20 -> monthly, 5 -> weekly
    sample_returns = is_returns # the sample returns serires keeps getting longer: after each iteration new informatuion is added
    os_decreasing = os_returns # out of sample returns, shrinking as sample increases
    forecasts_symbol = pd.DataFrame(index=os_data.index)
    horizons = (1, 5, 20)
    for p in range(len(horizons)):
        forecasts_symbol[symbol + f"_{horizons[p]}_h_sigma"] = np.zeros(len(os_returns))
        forecasts_symbol[symbol + f"_{horizons[p]}_h_omega"] = np.zeros(len(os_returns))
    #print(symbol + f"_{horizons[p]}_h")
    print(forecasts_symbol.columns)

    for i in range(int(len(os_returns)/update_window)):
        if i == 0:
            # different starting values for optimisation as well as for forecasting in the first iteration
            res_lstm = opt.minimize(RECH.LSTM_garch_loglike, vP0, args = (act_func, sample_returns),
                          bounds = my_bounds,
                           #method = "Nelder-Mead",
                            method = "SLSQP",
                          options = {"disp": False, "maxiter": 100000, "ftol": 0.1},
                           constraints = cons)
            # sample returns contain all information up to t = t
            # mc forecast function uses the lastest return as information
            
            for k in range(update_window):
                """for every time (t+1), (t+2), ... there are 3 forecasts made with respective information
                (t+1)-h1, (t+1)-h2, (t+1)-h3, (t+2)-h1, ... """
                for horizon in horizons:
                    fore_sigma = RECH.mc_lstm2(res_lstm.x, act_func, sample_returns[:-horizon], mc_M, horizon)[0]
                    fore_omega = RECH.mc_lstm2(res_lstm.x, act_func, sample_returns[:-horizon], mc_M, horizon)[1]
                    forecasts_symbol[symbol + f"_{horizon}_h_sigma"].iloc[i*update_window + k] = fore_sigma
                    forecasts_symbol[symbol + f"_{horizon}_h_omega"].iloc[i*update_window + k] = fore_omega
                    if fore_sig < 0:
                        warning_list.append(symbol)
                        warning_list.append(res_srn.x)
                ### now for every k the sample grows by 1
                sample_returns = sample_returns.append(os_decreasing.head(1))
                os_decreasing = os_decreasing.tail(-1)
                
        else:
            sample_returns = sample_returns.tail(-update_window)
            new_bounds = my_bounds
            res_lstm = opt.minimize(RECH.LSTM_garch_loglike, res_lstm.x, args = (act_func, sample_returns),
                          bounds = new_bounds,
                           #method = "Nelder-Mead",
                            method = "SLSQP",
                          options = {"disp": False, "maxiter": 30000, "ftol": 0.1},
                            constraints = cons)
            for k in range(update_window):
                """for every time (t+1), (t+2), ... there are 3 forecasts made with respective information
                (t+1)-h1, (t+1)-h2, (t+1)-h3, (t+2)-h1, ... """
                for horizon in horizons:
                    fore_sigma = RECH.mc_lstm2(res_lstm.x, act_func, sample_returns[:-horizon], mc_M, horizon)[0]
                    fore_omega = RECH.mc_lstm2(res_lstm.x, act_func, sample_returns[:-horizon], mc_M, horizon)[1]
                    forecasts_symbol[symbol + f"_{horizon}_h_sigma"].iloc[i*update_window + k] = fore_sigma
                    forecasts_symbol[symbol + f"_{horizon}_h_omega"].iloc[i*update_window + k] = fore_omega
                    if fore_sig < 0:
                        warning_list.append(symbol)
                        warning_list.append(res_srn.x)
                ### now for every k the sample grows by 1
                sample_returns = sample_returns.append(os_decreasing.head(1))
                os_decreasing = os_decreasing.tail(-1)
                #print(f"in sample: {len(sample_returns)}, out of sample: {len(os_decreasing)}")
    forecast_all = pd.concat([forecast_all, forecasts_symbol], axis=1)
    df_pars.iloc[my_list.index(symbol)] = res_lstm.x  
with open('lstm_warnings', 'wb') as fp:
    pickle.dump(warning_list, fp)

[*********************100%***********************]  1 of 1 completed
####################
Date
2012-01-04 00:00:00-05:00   -0.044893
2012-01-05 00:00:00-05:00   -0.420073
2012-01-06 00:00:00-05:00   -0.240788
2012-01-09 00:00:00-05:00    0.421052
Name: log_ret * 100, dtype: float64
####################
Index(['PG_1_h_sigma', 'PG_1_h_omega', 'PG_2_h_sigma', 'PG_2_h_omega'], dtype='object')


In [199]:
forecast_all

,PG_1_h_sigma,PG_1_h_omega,PG_2_h_sigma,PG_2_h_omega
Date,,,,
2017-01-03 00:00:00-05:00,0.646467,0.285623,0.464847,0.307302
2017-01-04 00:00:00-05:00,0.608758,0.237918,1.665656,0.654365
2017-01-05 00:00:00-05:00,0.595918,0.246900,0.826666,0.967940
2017-01-06 00:00:00-05:00,0.560800,0.231005,2.076522,0.359668
2017-01-09 00:00:00-05:00,0.540207,0.213986,0.475901,2.597965
...,...,...,...,...
2017-09-25 00:00:00-04:00,0.000000,0.000000,0.000000,0.000000
2017-09-26 00:00:00-04:00,0.000000,0.000000,0.000000,0.000000
2017-09-27 00:00:00-04:00,0.000000,0.000000,0.000000,0.000000
